In [2]:
from brigade.core import InitBrigade
brg = InitBrigade(config_file="config.yaml")

# Executing tasks

Now that you know how to initialize brigade and work with the inventory let's see how we can leverage it to run tasks on groups of hosts.

Brigade ships a bunch of tasks you can use directly without having to code them yourself. You can check them out [here](../../ref/tasks/index.rst).

Let's start by executing the `ls -la /tmp` command on all the device in `cmh` of type `host`:


In [3]:
from brigade.plugins.tasks import commands, text

cmh_hosts = brg.filter(site="cmh", role="host")

result = cmh_hosts.run(commands.remote_command,
                       command="ls -la /tmp")

cmh_hosts.run(text.print_result,
              num_workers=1,
              data=result,
              vars=["stdout"])

* host1.cmh ** changed : False *************************************************
---- remote_command ** changed : False  ----------------------------------------
total 8
drwxrwxrwt  2 root root 4096 Mar 11 19:17 .
drwxr-xr-x 24 root root 4096 Mar 11 17:28 ..


* host2.cmh ** changed : False *************************************************
---- remote_command ** changed : False  ----------------------------------------
total 8
drwxrwxrwt  2 root root 4096 Mar 11 19:17 .
drwxr-xr-x 24 root root 4096 Mar 11 17:29 ..




AggregatedResult: print_result

So what have we done here? First we have imported the `commands` and `text` modules. Then we have narrowed down brigade to the hosts we want to operate on. Once we have selected the devices we wanted to operate on we have run two tasks:

1. The task `commands.remote_command` which runs the specified `command` in the remote device.
2. The task `text.print_result`  which just prints on screen the result of an executed task (note this task needs to be serialized to avoid mangling the output so `num_workers=1` is specified).

Let's try with another example:

In [4]:
from brigade.plugins.tasks import networking

cmh_spines = brg.filter(site="bma", role="spine")
result = cmh_spines.run(networking.napalm_get,
                        getters=["facts"])                
cmh_spines.run(text.print_result,
              num_workers=1,
              data=result)

* spine00.bma ** changed : False ***********************************************
---- napalm_get ** changed : False  --------------------------------------------
{ 'facts': { 'fqdn': 'localhost',
             'hostname': 'localhost',
             'interface_list': ['Ethernet1', 'Ethernet2', 'Management1'],
             'model': 'vEOS',
             'os_version': '4.17.5M-4414219.4175M',
             'serial_number': '',
             'uptime': 66816,
             'vendor': 'Arista'}}

* spine01.bma ** changed : False ***********************************************
---- napalm_get ** changed : False  --------------------------------------------
{ 'facts': { 'fqdn': 'vsrx',
             'hostname': 'vsrx',
             'interface_list': [ 'ge-0/0/0',
                                 'gr-0/0/0',
                                 'ip-0/0/0',
                                 'lsq-0/0/0',
                                 'lt-0/0/0',
                                 'mt-0/0/0',
                

AggregatedResult: print_result

Pretty much the same pattern, just different task on different devices.

## What is a task

Let's take a look at what a task is. In it's simplest form a task is a function that takes at least a [Task](../../ref/api/task.rst#brigade.core.task.Task) object as argument. For instance:



In [5]:
def hi(task):
    print(f"hi! My name is {task.host.name} and I live in {task.host['site']}")
    
brg.run(hi, num_workers=1)

hi! My name is host1.cmh and I live in cmh
hi! My name is host2.cmh and I live in cmh
hi! My name is spine00.cmh and I live in cmh
hi! My name is spine01.cmh and I live in cmh
hi! My name is leaf00.cmh and I live in cmh
hi! My name is leaf01.cmh and I live in cmh
hi! My name is host1.bma and I live in bma
hi! My name is host2.bma and I live in bma
hi! My name is spine00.bma and I live in bma
hi! My name is spine01.bma and I live in bma
hi! My name is leaf00.bma and I live in bma
hi! My name is leaf01.bma and I live in bma


AggregatedResult: hi

The task object has access to `brigade`, `host` and `dry_run` attributes.

You can call other tasks from within a task:

In [6]:
def available_resources(task):
    task.run(commands.remote_command,
             name="Available disk",
             command="df -h")
    task.run(commands.remote_command,
             name="Available memory",
             command="free -m")
    
result = cmh_hosts.run(available_resources)

cmh_hosts.run(text.print_result,
              num_workers=1,
              data=result,
              vars=["stdout"])

* host1.cmh ** changed : False *************************************************
---- available_resources ** changed : False  -----------------------------------
---- Available disk ** changed : False  ----------------------------------------
Filesystem                  Size  Used Avail Use% Mounted on
/dev/mapper/precise64-root   79G  2.2G   73G   3% /
udev                        174M  4.0K  174M   1% /dev
tmpfs                        74M  284K   73M   1% /run
none                        5.0M     0  5.0M   0% /run/lock
none                        183M     0  183M   0% /run/shm
/dev/sda1                   228M   25M  192M  12% /boot
vagrant                     373G  200G  173G  54% /vagrant

---- Available memory ** changed : False  --------------------------------------
             total       used       free     shared    buffers     cached
Mem:           365         87        277          0          8         36
-/+ buffers/cache:         42        323
Swap:          767          0

AggregatedResult: print_result

You probably noticed in your previous example that you can name your tasks.

Your task can also accept any extra arguments you may need:

In [7]:
def count(task, to):
    print(f"{task.host.name}: {list(range(0, to))}")
    
cmh_hosts.run(count,
              num_workers=1,
              to=10)
cmh_hosts.run(count,
              num_workers=1,
              to=20)

host1.cmh: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
host2.cmh: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
host1.cmh: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
host2.cmh: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


AggregatedResult: count